<center>
<img src="../../img/ods_stickers.jpg">
## Открытый курс по машинному обучению. Сессия № 2
Автор материала: программист-исследователь Mail.ru Group, старший преподаватель Факультета Компьютерных Наук ВШЭ Юрий Кашницкий. Материал распространяется на условиях лицензии [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/). Можно использовать в любых целях (редактировать, поправлять и брать за основу), кроме коммерческих, но с обязательным упоминанием автора материала.

# <center>Домашнее задание № 10
## <center> Прогнозирование задержек вылетов

Ваша задача – побить как минимум 2 бенчмарка в [соревновании](https://www.kaggle.com/c/flight-delays-2017) на Kaggle Inclass. Подробных инструкций не будет, будет только тезисно описано, как получен второй – с помощью Xgboost. Надеюсь, на данном этапе курса вам достаточно бросить полтора взгляда на данные, чтоб понять, что это тот тип задачи, в которой затащит градиентный бустинг. Скорее всего Xgboost, но тут у нас еще немало категориальных признаков...

<img src='../../img/xgboost_meme.jpg' width=40% />

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
import xgboost as xgb

In [2]:
train = pd.read_csv('../../data/hw10_xgboost/flight_delays_train.csv')
test = pd.read_csv('../../data/hw10_xgboost/flight_delays_test.csv')

Итак, надо по времени вылета самолета, коду авиакомпании-перевозчика, месту вылета и прилета и расстоянию между аэропортами вылета и прилета предсказать задержку вылета более 15 минут. В качестве простейшего бенчмарка возьмем логистическую регрессию и два признака, которые проще всего взять: `DepTime` и `Distance`. У такой модели результат – 0.68202 на LB. 

In [3]:
X_train, y_train = train[['Distance', 'DepTime']].values, train['dep_delayed_15min'].map({'Y': 1, 'N': 0}).values
X_test = test[['Distance', 'DepTime']].values

X_train_part, X_valid, y_train_part, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=17)

scaler = StandardScaler()
X_train_part = scaler.fit_transform(X_train_part)
X_valid = scaler.transform(X_valid)

/usr/local/lib/python3.5/dist-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)


In [4]:
logit = LogisticRegression()

logit.fit(X_train_part, y_train_part)
logit_valid_pred = logit.predict_proba(X_valid)[:, 1]

roc_auc_score(y_valid, logit_valid_pred)

0.67956914653526068

In [5]:
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

logit.fit(X_train_scaled, y_train)
logit_test_pred = logit.predict_proba(X_test_scaled)[:, 1]

pd.Series(logit_test_pred, name='dep_delayed_15min').to_csv('logit_2feat.csv', index_label='id', header=True)

/usr/local/lib/python3.5/dist-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)


Второй бенчмарк, представленный в рейтинге соревнования, был получен так:
- Признаки `Distance` и  `DepTime` брались без изменений
- Создан признак "маршрут" из исходных `Origin` и `Dest`
- К признакам `Month`, `DayofMonth`, `DayOfWeek`, `UniqueCarrier` и "маршрут" применено OHE-преобразование (`LabelBinarizer`)
- Выделена отложенная выборка
- Обучалась логистическая регрессия и градиентный бустинг (xgboost), гиперпараметры бустинга настраивались на кросс-валидации, сначала те, что отвечают за сложность модели, затем число деревьев фиксировалось равным 500 и настраивался шаг градиентного спуска
- С помощью `cross_val_predict` делались прогнозы обеих моделей на кросс-валидации (именно предсказанные вероятности), настраивалась линейная смесь ответов логистической регрессии и градиентного бустинга вида $w_1 * p_{logit} + (1 - w_1) * p_{xgb}$, где $p_{logit}$ – предсказанные логистической регрессией вероятности класса 1, $p_{xgb}$ – аналогично. Вес $w_1$ подбирался вручную. 
- В качестве ответа для тестовой выборки бралась аналогичная комбинация ответов двух моделей, но уже обученных на всей обучающей выборке.

Описанный план ни к чему не обязывает – это просто то, как решение получил автор задания. Возможно, вы не захотите следовать намеченному плану, а добавите, скажем, пару хороших признаков и обучите лес из тысячи деревьев.

Удачи!

In [6]:
from sklearn.preprocessing import LabelBinarizer
from scipy.sparse import hstack

In [7]:
%%time

train_new = train.copy()
test_new = test.copy()

train_new["Route"] = train_new["Origin"] + "_" + train_new["Dest"]
test_new["Route"] = test_new["Origin"] + "_" + test_new["Dest"]

lb = LabelBinarizer(sparse_output=True)

route_train = lb.fit_transform(train_new["Route"])
route_test = lb.transform(test_new["Route"])

month_train = lb.fit_transform(train_new["Month"])
month_test = lb.transform(test_new["Month"])

day_of_month_train = lb.fit_transform(train_new["DayofMonth"])
day_of_month_test = lb.transform(test_new["DayofMonth"])

day_of_week_train = lb.fit_transform(train_new["DayOfWeek"])
day_of_week_test = lb.transform(test_new["DayOfWeek"])

uniq_carrier_train = lb.fit_transform(train_new["UniqueCarrier"])
uniq_carrier_test = lb.transform(test_new["UniqueCarrier"])

# train_new[["bin_" + cls for cls in lb.classes_]] = pd.DataFrame(lb.transform(train_new["Route"]))
# test_new[["bin_" + cls for cls in lb.classes_]] = pd.DataFrame(lb.transform(test_new["Route"]))

# lb.fit(train_new["Month"])
# train_new[["bin_" + cls for cls in lb.classes_]] = pd.DataFrame(lb.transform(train_new["Month"]))
# test_new[["bin_" + cls for cls in lb.classes_]] = pd.DataFrame(lb.transform(test_new["Month"]))

# lb.fit(train_new["DayofMonth"])
# train_new[["bin_" + cls for cls in lb.classes_]] = pd.DataFrame(lb.transform(train_new["DayofMonth"]))
# test_new[["bin_" + cls for cls in lb.classes_]] = pd.DataFrame(lb.transform(test_new["DayofMonth"]))

# lb.fit(train_new["DayOfWeek"])
# train_new[["bin_" + cls for cls in lb.classes_]] = pd.DataFrame(lb.transform(train_new["DayOfWeek"]))
# test_new[["bin_" + cls for cls in lb.classes_]] = pd.DataFrame(lb.transform(test_new["DayOfWeek"]))

# lb.fit(train_new["UniqueCarrier"])
# train_new[["bin_" + cls for cls in lb.classes_]] = pd.DataFrame(lb.transform(train_new["UniqueCarrier"]))
# test_new[["bin_" + cls for cls in lb.classes_]] = pd.DataFrame(lb.transform(test_new["UniqueCarrier"]))

CPU times: user 3.86 s, sys: 72 ms, total: 3.93 s
Wall time: 3.96 s


In [8]:
X_train_new = hstack([X_train_scaled, route_train, month_train, day_of_month_train, day_of_week_train, uniq_carrier_train])
X_test_new = hstack([X_test_scaled, route_test, month_test, day_of_month_test, day_of_week_test, uniq_carrier_test])

X_train_new.shape, X_test_new.shape

((100000, 4503), (100000, 4503))

In [9]:
X_train_part, X_valid, y_train_part, y_valid = train_test_split(X_train_new, y_train, test_size=0.3, random_state=17)

In [10]:
from hyperopt import hp
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

In [38]:
def score(params):
    from sklearn.metrics import log_loss
    print("Training with params:")
    print(params)
    params['max_depth'] = int(params['max_depth'])
    dtrain = xgb.DMatrix(X_train_part, label=y_train_part)
    dvalid = xgb.DMatrix(X_valid, label=y_valid)
    model = xgb.train(params, dtrain, params['num_round'])
    predictions = model.predict(dvalid)
    score = log_loss(y_valid, predictions)
    print("\tScore {0}\n\n".format(score))
    return {'loss': score, 'status': STATUS_OK}

In [67]:
def optimize(trials):
    space = {
             'num_round': 100,
             'learning_rate': hp.quniform('eta', 0.005, 0.05, 0.005),
             'max_depth': hp.quniform('max_depth', 3, 18, 1),
             'min_child_weight': hp.quniform('min_child_weight', 1, 10, 1),
             'subsample': hp.quniform('subsample', 0.5, 1, 0.05),
             'gamma': hp.quniform('gamma', 0.5, 1, 0.01),
             'objective': 'binary:logistic',
             'nthread' : 4,
             'silent' : 1
             }
    
    best = fmin(score, space, algo=tpe.suggest, trials=trials, max_evals=10)
    return best

In [68]:
trials = Trials()
best_params = optimize(trials)
best_params

Training with params:
{'min_child_weight': 6.0, 'silent': 1, 'learning_rate': 0.005, 'nthread': 4, 'objective': 'binary:logistic', 'gamma': 0.68, 'num_round': 100, 'max_depth': 3.0, 'subsample': 0.8}
	Score 0.5534719979981582


Training with params:
{'min_child_weight': 2.0, 'silent': 1, 'learning_rate': 0.03, 'nthread': 4, 'objective': 'binary:logistic', 'gamma': 0.5700000000000001, 'num_round': 100, 'max_depth': 7.0, 'subsample': 0.65}
	Score 0.4368425001559779


Training with params:
{'min_child_weight': 6.0, 'silent': 1, 'learning_rate': 0.035, 'nthread': 4, 'objective': 'binary:logistic', 'gamma': 0.89, 'num_round': 100, 'max_depth': 11.0, 'subsample': 0.9}
	Score 0.43229486943936596


Training with params:
{'min_child_weight': 7.0, 'silent': 1, 'learning_rate': 0.015, 'nthread': 4, 'objective': 'binary:logistic', 'gamma': 0.89, 'num_round': 100, 'max_depth': 8.0, 'subsample': 0.8500000000000001}
	Score 0.4598031818434596


Training with params:
{'min_child_weight': 4.0, 'silent':

{'eta': 0.05,
 'gamma': 0.56,
 'max_depth': 14.0,
 'min_child_weight': 1.0,
 'subsample': 0.55}

In [69]:
best_params['max_depth'] = int(best_params['max_depth'])
best_params['objective'] = 'binary:logistic'
best_params['nthread'] = 4
best_params['silent'] = 1

In [70]:
dtrain = xgb.DMatrix(X_train_new, y_train)

In [71]:
%%time
xgbCvResult = xgb.cv(best_params, dtrain, 
                      num_boost_round=500,  
                      nfold=3, early_stopping_rounds=50)

CPU times: user 5min 13s, sys: 6.07 s, total: 5min 19s
Wall time: 1min 24s


In [72]:
best_num_round = np.argmin(xgbCvResult['test-error-mean'])
best_num_round

224

In [73]:
bestXgb = xgb.train(best_params, dtrain, num_boost_round=best_num_round)

In [74]:
dtest = xgb.DMatrix(X_test_new)

In [75]:
xgboost_predict_proba = bestXgb.predict(dtest)

In [76]:
pd.Series(xgboost_predict_proba, name='dep_delayed_15min').to_csv('xgboost3.csv', index_label='id', header=True)